In [1]:
import sys
sys.path.append('../')

from inference import Inference, get_xray_classes
import pandas as pd
import os

/data/ephemeral/conda_envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/data/ephemeral/conda_envs/torch/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [4]:
prefix = "1125"
mode = 'valid'
crop_types = ['arm', 'finger', 'backhand']
fold_name_format = 'ensemble_crop_{crop_type}_TTA_False_2-models'


os.makedirs('result', exist_ok=True)
save_path = f'result/{prefix}_{mode}_' + fold_name_format.replace('{crop_type}_', "") + ".csv"

file_paths = []
for crop_type in crop_types:
    fold_name = fold_name_format.format(crop_type=crop_type)
    file_path = f'{fold_name}/{mode}_{fold_name}.csv'
    file_paths.append(file_path)

df = None
classes = get_xray_classes()

for file_path in file_paths:
    if df is None:
        df = pd.read_csv(file_path)
    else:
        df = pd.concat([df, pd.read_csv(file_path)])

df['class_num'] = df['class'].map(classes['class2idx'])
df = df.sort_values(by=['image_name', 'class_num'])
df = df.drop(columns=['class_num'])

df.to_csv(save_path, index=False)